In [1]:
import os
import cv2
import dlib
import base64
import pickle
import sqlite3
import datetime
import face_recognition
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
models_dir = os.path.join(os.path.dirname(
    os.getcwd()), "project", "base", "models")
prototxtPath = os.path.join(
    models_dir, "Resnet_SSD_deploy.prototxt")
weightsPath = os.path.join(
    models_dir, "Res10_300x300_SSD_iter_140000.caffemodel")
print(models_dir)
faceNet = cv2.dnn.readNetFromCaffe(
    prototxtPath, weightsPath)
db_dir = os.path.join(os.path.dirname(
    os.getcwd()), "project", "db.sqlite3")

F:\Face AMS\project\base\models


In [3]:
def decode(new_encoding):
    np_bytes = base64.b64decode(new_encoding)
    np_array = pickle.loads(np_bytes)
    return np_array[0]

In [4]:
conn = sqlite3.connect(db_dir)
students = pd.read_sql(f"SELECT * FROM school_student WHERE study_class_id == {1}", conn)

In [5]:
students

,student_id,contact,address,face_encoding,study_class_id,user_id,profile_image
0,3921,9825953356,jhapa,b'gASVjgQAAAAAAABdlIwVbnVtcHkuY29yZS5tdWx0aWFy...,1,48,profile_images/3921_face.jpg
1,12345,214124,41241,b'gASVjgQAAAAAAABdlIwVbnVtcHkuY29yZS5tdWx0aWFy...,1,49,profile_images/12345_face.jpg
2,18030073,9867719666,"Butwal, Nepal",b'gASVjgQAAAAAAABdlIwVbnVtcHkuY29yZS5tdWx0aWFy...,1,44,profile_images/default_Mqs7bdZ.jpg
3,18030074,9867719666,"Butwal, Nepal",b'gASVjgQAAAAAAABdlIwVbnVtcHkuY29yZS5tdWx0aWFy...,1,43,profile_images/me_FFQASU4.jpg


In [6]:
students["face_encoding"] = students["face_encoding"].apply(decode)

In [7]:
students

,student_id,contact,address,face_encoding,study_class_id,user_id,profile_image
0,3921,9825953356,jhapa,"[-0.19450820982456207, 0.07507634162902832, 0....",1,48,profile_images/3921_face.jpg
1,12345,214124,41241,"[-0.16336506605148315, 0.08043473958969116, 0....",1,49,profile_images/12345_face.jpg
2,18030073,9867719666,"Butwal, Nepal","[-0.09007669240236282, 0.053269874304533005, 0...",1,44,profile_images/default_Mqs7bdZ.jpg
3,18030074,9867719666,"Butwal, Nepal","[-0.12828867137432098, 0.06153010576963425, 0....",1,43,profile_images/me_FFQASU4.jpg


In [8]:
users = pd.read_sql(f"SELECT * FROM account_user WHERE id in ({students['user_id'].tolist()})".replace('[','').replace(']', ''), conn)

In [9]:
users["user_id"] = users["id"]
users

,id,password,last_login,email,full_name,is_staff,is_active,is_admin,user_id
0,43,pbkdf2_sha256$216000$Cud7LVBXGSZQ$u+g2REexrNsQ...,2021-04-14 08:04:18.231865,ajeetkharell@gmail.com,Amit Kharel,0,1,0,43
1,44,pbkdf2_sha256$216000$oxkMznNSjahq$AiRGeua7NpsY...,2021-03-27 01:58:23.249930,mramitkharel@gmail.com,Ajeet Kharel,0,1,0,44
2,48,pbkdf2_sha256$216000$m3XebTbbqx6t$DDfona/Zhrih...,None,adhikarisaroj291@gmail.com,Saroj Adhikari,0,1,0,48
3,49,pbkdf2_sha256$216000$9h7SpaSWKi7C$pK8bb+ELPtHY...,None,arnavshresthaneywar@gmail.com,Arnav Shrestha,0,1,0,49


In [10]:
merged = pd.merge(students, users, on='user_id')[["student_id", "face_encoding", "full_name"]]

In [11]:
merged

,student_id,face_encoding,full_name
0,3921,"[-0.19450820982456207, 0.07507634162902832, 0....",Saroj Adhikari
1,12345,"[-0.16336506605148315, 0.08043473958969116, 0....",Arnav Shrestha
2,18030073,"[-0.09007669240236282, 0.053269874304533005, 0...",Ajeet Kharel
3,18030074,"[-0.12828867137432098, 0.06153010576963425, 0....",Amit Kharel


In [12]:
def get_nearest_student(b):
    def get_sim(a):
        return np.dot(a, b[0])/(np.linalg.norm(a)*np.linalg.norm(b[0]))
    
    return merged.iloc[merged["face_encoding"].apply(get_sim).idxmax()]

In [13]:
routines = pd.read_sql(f"SELECT * FROM school_routine WHERE _class_id = {1}", conn)

In [14]:
routines

,id,_class_id,subject_id,end_time,start_time
0,2,1,1,17:08:44,15:08:44
1,3,1,2,15:51:13,12:30:50


In [54]:
def check_routine(time, date):
    for index, row in routines.iterrows():
        start_time = datetime.datetime.strptime(row["start_time"], "%H:%M:%S").time()
        end_time = datetime.datetime.strptime(row["end_time"], "%H:%M:%S").time()
        
        if time >= start_time and time <= end_time:
            return (row["id"],
                    datetime.datetime.combine(date, start_time).strftime('%Y-%m-%d %H:%M:%S'), 
                    datetime.datetime.combine(date, end_time).strftime('%Y-%m-%d %H:%M:%S'))
        
    return None, None, None

In [55]:
def attendance_exists(attendance, start_time, end_time):
    result = pd.read_sql(f"SELECT * FROM school_attendance WHERE \
                routine_id={attendance[0]} AND \
                student_id = {attendance[1]} AND \
                date BETWEEN '{start_time}' AND '{end_time}'", conn)
    return len(result) != 0

In [56]:
attendances = pd.read_sql(f"SELECT * FROM school_attendance WHERE routine_id in ({routines['id'].tolist()})".replace('[','').replace(']', ''), conn)

In [57]:
attendances

,id,routine_id,student_id,status,date


In [58]:
cap = cv2.VideoCapture(0)
faces = []
while True:
    ret, frame = cap.read()
    if ret:
        (h, w) = frame.shape[:2]
        frame =  cv2.flip(frame, 1)
        blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),
            (104.0, 177.0, 123.0))
        faceNet.setInput(blob)
        detections = faceNet.forward()

        for i in range(0, detections.shape[2]):
            confidence = detections[0, 0, i, 2]

            if confidence > 0.5:
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")

                (startX, startY) = (max(0, startX), max(0, startY))
                (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
                
                faceimage = frame[startY:endY, startX:endX]
                faceimage = faceimage[:, :, ::-1]

                faces.append(faceimage)
                
                face_encodings = face_recognition.face_encodings(faceimage)
                if face_encodings:
                    student_details = get_nearest_student(face_encodings)
                    date = datetime.datetime.now()
                    current_time = date.time()
                    current_datetime = date.strftime('%Y-%m-%d %H:%M:%S')
                    routine, start_time, end_time = check_routine(current_time, date.date())
                    if routine is not None:
                        attendance = (routine, int(student_details["student_id"]), 'P', current_datetime)
                        if not attendance_exists(attendance, start_time, end_time):
                            sql = ''' INSERT INTO school_attendance(routine_id, student_id, status, date)
                                          VALUES(?,?,?,?) '''
                            cur = conn.cursor()
                            cur.execute(sql, attendance)
                            conn.commit()
                    
                    cv2.rectangle(frame, (startX, startY),
                                  (endX, endY), (255, 255, 0), 2)
                    cv2.putText(frame, student_details["full_name"], (startX, startY-40), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
                    cv2.putText(frame, str(student_details["student_id"]), (startX, startY-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
        cv2.imshow("Face AMS", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
cap.release()
cv2.destroyAllWindows()